In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# **Problelm 1**
## **Apply all the processing techniques that you think are necessary**

## **1. Lowercase**

In [4]:
df['review'].str.lower()

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [5]:
df['review'] = df['review'].str.lower()

## **2. Remove HTML**

In [6]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

In [7]:
df['review'].apply(remove_html_tags)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [8]:
df['review'] = df['review'].apply(remove_html_tags)

## **3. Remove URLs** 
### *URL is not available in this data. So, we can skip this step.*

## **Remove link**

In [9]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [10]:
df['review'].apply(remove_url)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [11]:
df['review'] = df['review'].apply(remove_url)

## **4. Remove Punctuation**
### *removing punctuation if available in data*

In [12]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
punc = string.punctuation

In [14]:
def remove_punc1(text):
    return text.translate(str.maketrans('', '', punc))

### **In str.maketrans('', '', punc), the three arguments serve the following purposes:**

First argument (''): This is for character replacements (not used here).<br>
Second argument (''): This is for character substitutions (also not used here). <br>
Third argument (punc): This specifies characters to be removed from the text. <br>
### **Why use '', '', punc?** <br>
The first two empty strings ('', '') mean we are not replacing or mapping any characters. <br>
The third argument (punc) contains all punctuation characters, telling translate() to remove them from the text.

In [15]:
df['review'].apply(remove_punc1)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production the filming tech...
2        i thought this was a wonderful way to spend ti...
3        basically theres a family where a little boy j...
4        petter matteis love in the time of money is a ...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    i am a catholic taught in parochial elementary...
49998    im going to have to disagree with the previous...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [16]:
df['review'][3]

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

## **5. Rewrite Shortcut Chat Word**

In [17]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek You (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your Sex and Age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That Feeling When",
    "MFW": "My Face When",
    "MRW": "My Reaction When",
    "IFYP": "I Feel Your Pain",
    "LOL": "Laughing Out Loud",
    "TNTL": "Trying Not To Laugh",
    "JK": "Just Kidding",
    "IDC": "I Don't Care",
    "ILY": "I Love You",
    "IMU": "I Miss You",
    "ADIH": "Another Day In Hell",
    "ZZZ": "Sleeping, Bored, Tired",
    "WYWH": "Wish You Were Here",
    "TIME": "Tears In My Eyes",
    "BAE": "Before Anyone Else",
    "FIMH": "Forever In My Heart",
    "BSAAW": "Big Smile And A Wink",
    "BWL": "Bursting With Laughter",
    "LMAO": "Laughing My A** Off",
    "BFF": "Best Friends Forever",
    "CSL": "Can't Stop Laughing"
}

In [18]:
def chat_conversion(text):
    return " ".join(chat_words.get(w.upper(), w) for w in text.split())

In [19]:
df['review'].apply(chat_conversion)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend Te...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the Tears In My Eyes ...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [20]:
df['review'] = df['review'].apply(chat_conversion)

### **6. Remove stop words**

In [21]:
from nltk.corpus import stopwords

In [22]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [23]:
top_words = set(stopwords.words('english'))  

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in top_words])

In [24]:
df['review'].apply(remove_stopwords)

0        one reviewers mentioned watching 1 oz episode ...
1        wonderful little production. filming technique...
2        thought wonderful way spend Tears In My Eyes h...
3        basically there's family little boy (jake) thi...
4        petter mattei's "love Tears In My Eyes money" ...
                               ...                        
49995    thought movie right good job. creative origina...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    catholic taught parochial elementary schools n...
49998    i'm going disagree previous comment side malti...
49999    one expects star trek movies high art, fans ex...
Name: review, Length: 50000, dtype: object

In [25]:
df['review'] = df['review'].apply(remove_stopwords)

In [26]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [27]:
df['review'].apply(stem_words)

0        one review mention watch 1 oz episod hooked. r...
1        wonder littl production. film techniqu unassum...
2        thought wonder way spend tear In My eye hot su...
3        basic there' famili littl boy (jake) think the...
4        petter mattei' "love tear In My eye money" vis...
                               ...                        
49995    thought movi right good job. creativ origin fi...
49996    bad plot, bad dialogue, bad acting, idiot dire...
49997    cathol taught parochi elementari school nuns, ...
49998    i'm go disagre previou comment side maltin one...
49999    one expect star trek movi high art, fan expect...
Name: review, Length: 50000, dtype: object

In [28]:
df['review '] = df['review'].apply(stem_words)

# Problem 2

## *Find out the number of words in the entire corpus and also the total number of unique words(vocabulary) using just python*

### **Number of Words**

In [29]:
from collections import Counter

# Merge all text and tokenize
all_text = " ".join(df['review']).lower()
word_counts = Counter(all_text.split())

# Total words (including duplicates)
total_words = sum(word_counts.values())

print("Total words in corpus:", total_words)

Total words in corpus: 6204825


In [30]:
word_counts

Counter({'one': 45067,
         'reviewers': 418,
         'mentioned': 810,
         'watching': 8155,
         '1': 1249,
         'oz': 129,
         'episode': 2213,
         'hooked.': 55,
         'right,': 436,
         'exactly': 1861,
         'happened': 1489,
         'me.the': 60,
         'first': 15475,
         'thing': 7113,
         'struck': 257,
         'brutality': 111,
         'unflinching': 28,
         'scenes': 7799,
         'violence,': 416,
         'set': 3968,
         'right': 4646,
         'word': 1319,
         'go.': 288,
         'trust': 505,
         'me,': 2136,
         'show': 9375,
         'faint': 71,
         'hearted': 104,
         'timid.': 3,
         'pulls': 370,
         'punches': 88,
         'regards': 112,
         'drugs,': 172,
         'sex': 2371,
         'violence.': 258,
         'hardcore,': 8,
         'classic': 2552,
         'use': 3383,
         'word.it': 2,
         'called': 2511,
         'nickname': 37,
        

### **Number of Unique Words**

In [31]:
# Merge all text and tokenize
all_text = " ".join(set(df['review'])).lower()
word_counts = Counter(all_text.split())

# Total words (including duplicates)
total_words = sum(word_counts.values())

print("Total words in corpus:", total_words)

Total words in corpus: 6157525


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review'])

# Get unique words
unique_words = vectorizer.get_feature_names_out()

print("Unique words in dataset:", unique_words)
print("Total unique words:", len(unique_words))


Unique words in dataset: ['00' '000' '00000000000' ... 'żmijewski' 'יגאל' 'כרמון']
Total unique words: 103871


In [33]:
unique_words[1]

'000'

In [34]:
from collections import Counter

# Merge all text, lowercase, and split
all_text = " ".join(df['review']).lower().split()
word_counts = Counter(all_text)

# Get unique words
unique_words = set(word_counts.keys())

print("Total unique words:", len(unique_words))

Total unique words: 422124


## **CountVectorizer (103,871 unique words)**
It removes punctuation and considers only words. <br>
It ignores stopwords (if stop_words='english' is used). <br>
It doesn't count numbers or special characters unless specified.<br>
It tokenizes words differently (e.g., don't → dont or do not).<br>
## **Counter (422,124 unique words)**
It counts every token directly from splitting text on spaces.<br>
It doesn’t remove punctuation or stopwords.<br>
Words like "hello," and "hello" are counted separately.<br>
If data contains numbers, they are counted as words.<br>

# **Problem 3**

### *Apply One Hot Encoding*

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df['review'])

# Convert to DataFrame for better visualization
one_hot_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

print(one_hot_df)

       00  000  00000000000  0000000000001  00000001  00001  00015  000dm  \
0       0    0            0              0         0      0      0      0   
1       0    0            0              0         0      0      0      0   
2       0    0            0              0         0      0      0      0   
3       0    0            0              0         0      0      0      0   
4       0    0            0              0         0      0      0      0   
...    ..  ...          ...            ...       ...    ...    ...    ...   
49995   0    0            0              0         0      0      0      0   
49996   0    0            0              0         0      0      0      0   
49997   0    0            0              0         0      0      0      0   
49998   0    0            0              0         0      0      0      0   
49999   0    0            0              0         0      0      0      0   

       000s  001  ...  ünel  ünfaithful  üvegtigris  üzümcü  ýs  þorleifsso

In [36]:
one_hot_df

,00,000,00000000000,0000000000001,00000001,00001,00015,000dm,000s,001,...,ünel,ünfaithful,üvegtigris,üzümcü,ýs,þorleifsson,þór,żmijewski,יגאל,כרמון
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# **Problem 4**

### *Apply bag words and find the vocabulary also find the times each word has occured*

In [39]:
bow[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [40]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review'])

# Convert to DataFrame for better visualization
bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [41]:
bow

,00,000,00000000000,0000000000001,00000001,00001,00015,000dm,000s,001,...,ünel,ünfaithful,üvegtigris,üzümcü,ýs,þorleifsson,þór,żmijewski,יגאל,כרמון
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Problem 5

### *Apply bag of bi-gram and bag of tri-gram and write down your observation about the dimensionality of the vocabulary*

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
bg = cv.fit_transform(df['review'])